# Understand dag in spark web ui

In [5]:
from pyspark.sql import SparkSession
import os
from pyspark.sql.functions import col

In [2]:
local = True

if local:
    spark = SparkSession.builder \
        .master("local[4]") \
        .appName("UnderstandSparkUIDAG") \
        .getOrCreate()
else:
    spark = SparkSession.builder \
        .master("k8s://https://kubernetes.default.svc:443") \
        .appName("UnderstandSparkUIDAG") \
        .config("spark.kubernetes.container.image", "inseefrlab/jupyter-datascience:py3.9.7-spark3.2.0")\
        .config("spark.kubernetes.authenticate.driver.serviceAccountName", os.environ['KUBERNETES_SERVICE_ACCOUNT'])\
        .config("spark.executor.instances", "4")\
        .config("spark.executor.memory", "8g")\
        .config("spark.kubernetes.namespace", os.environ['KUBERNETES_NAMESPACE'])\
        .getOrCreate()

22/03/01 17:20:23 WARN Utils: Your hostname, pliu-SATELLITE-P850 resolves to a loopback address: 127.0.1.1; using 172.22.0.33 instead (on interface wlp3s0)
22/03/01 17:20:23 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/01 17:20:35 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
df=spark.range(1,100000)
df.show()

+---+
| id|
+---+
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
| 20|
+---+
only showing top 20 rows



In [7]:
df.select((col("id")*5).alias("id")).show()

+---+
| id|
+---+
|  5|
| 10|
| 15|
| 20|
| 25|
| 30|
| 35|
| 40|
| 45|
| 50|
| 55|
| 60|
| 65|
| 70|
| 75|
| 80|
| 85|
| 90|
| 95|
|100|
+---+
only showing top 20 rows

